# Final Project

Візуалізації рівня захворюваності на Covid-19.

1) Сумарна кількість випадків захворювання у світі на карті. + Графік зміни денної кількості випадків за часом для окремої країни.

2) Інтерактивний графік який показує як змінювалася кількість нових випадків зараження кожного дня впродовж 2-ох років.

3) Кількість випадків зараження залежно від регіону ВООЗ (всього 6 регіонів) та місяця. 

4) Рівень захворюваності в Україна у розрізі місяців та років.

Для проету використовувався датасет з сайту WHO : https://covid19.who.int/info/

In [ ]:
import pandas as pd
import altair as alt
import geopandas as gpd

alt.data_transformers.enable('default', max_rows=None)

In [ ]:
world = gpd.read_file('data/world-countries.json')
world.head()

In [ ]:
df = pd.read_csv('data/WHO-COVID-19-global-data.csv')
df.head()

 Дані містять 8 колонок :
 - Date_reported : дата отримання інформації ВООЗ,  
 - Country_code : код країни в форматі Alpha-2,  
 - Country : назва країни,  
 - WHO_region : регіональний офіс ВООЗ, "WHO Member States are grouped into six WHO regions -- 
     - Regional Office for Africa (AFRO), 
     - Regional Office for the Americas (AMRO), 
     - Regional Office for South-East Asia (SEARO), 
     - Regional Office for Europe (EURO), 
     - Regional Office for the Eastern Mediterranean (EMRO), 
     - and Regional Office for the Western Pacific (WPRO)."
 - New_cases : кількість випадків захворюваності за добу,  
 - Cumulative_cases : сукупна кількість випадків за весь час до вказаної дати включно,  
 - New_deaths : кількість смертельних випадків за одну добу,  
 - Cumulative_death : сукупна кількість відома на дане число. 

In [ ]:
df.drop(['New_deaths', 'Cumulative_deaths'], axis = 1, inplace = True)

In [ ]:
contry_codes = pd.read_csv('data/country-codes.csv')
contry_codes.head()

In [ ]:
contry_codes.drop(['country', 'numeric'], axis = 1, inplace = True)
contry_codes.columns = ['Country_code', 'id']

contry_codes.head()

In [ ]:
df = df.merge(contry_codes)
df.head()

In [ ]:
df.drop(['Country_code'], axis = 1, inplace = True)
df.columns = ['Date', 'Country', 'WHO_region', 'New_cases', 'Cumulative_cases', 'id']
df.head()

In [ ]:
df['Date'] = df['Date'].astype('datetime64[ns]')
df['Year']=df.Date.dt.year

In [ ]:
df.info()

В колоці `New_cases`, котра відповідає кількості нових випадків за день були присутні значення > 0. Можливо малося на увазі, скільки людей одужало. Я замінила від'ємні значення на нуль, будемо вважати що, якщо значення від'ємне то в цей день не було випадків захворюваності. 

In [ ]:
df.loc[df['New_cases'] < 0, 'New_cases'] = 0
df.head()

## (1) Сумарна кількість випадків захворювання у світі на карті. + Графік зміни денної кількості випадків за часом для окремої країни.

На карті буде зображена сумарна кількість випадків відома на 10.12.2021 (остання дата у датасеті).

In [ ]:
df_total_cases = df[df.Date == '2021-12-10']
df_total_cases.head()

Кількість випадків захворювання на карті передається кольором, від блідо жовтого до червоного. Червоний колір добре підходить щоб передати негативне зростання. 

Використовуючи звичайну лінійну Legend, буде важко зрозуміти якому числовому інтервалу відмовідає певний відтінок. Тому я використала alt.Bin, щоб розділити діапазон кольорів на окремі інтервали яким будуть відповідати певні числові значення. Я старалася підібрати таке розбиття, щоб деталізація не постраждала.

Мінусом даної візуалізаці можуть бути пропуски на карті для пд. Судану і для Анарктиди, для них відсутні дані в датасеті. 

Ще одним мінусом є неточності у датасеті. Наприклад у Китаї сумарна кількість випадків відносно низька. Я думаю це пов'язано з тим, що не всі країни надавали свої дані ВООЗ.

In [ ]:
map_total_cases = alt.Chart(world, 
          title={"text": ["Total cases of Covid-19 infections due to 10.12.2021"], 
                 "subtitle": ["hover over the required area for more information"],
                 "subtitleColor": '#333333'}
).transform_lookup(
    lookup = 'id',
    from_ = alt.LookupData(data = df_total_cases, 
                           key = 'id',
                           fields=['Cumulative_cases', 'Country']),
    default = 'null'
).project(type = 'equalEarth'
).mark_geoshape(stroke = 'black').encode(
    color = alt.Color('Cumulative_cases:Q', bin = alt.Bin(step = 5000000), title = 'number of cases', 
                      scale = alt.Scale(
                          scheme = 'lightorange',
                      ),
                      legend = alt.Legend(
                        format = '.1s',)
                     ),   
    tooltip = [
        alt.Tooltip('name:N', title = 'country'),
        alt.Tooltip('Cumulative_cases:N', title = 'number of cases'),        
    ]
)

map_total_cases.properties(width = 800, height = 500, background = '#F9F9F9', padding = 25
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 14,
                   gradientLength=350,
                   orient = 'bottom'
).configure_title(
    fontSize=20,
    font='Times',
).configure_view(strokeWidth = 0)

Наступний графік буде показувати денну кількість випадків впродовж двох років. 

Щоб стовпчики не накладалися один на одного я задала їм маленький size.

In [ ]:
line_chart_total = alt.Chart(df,
          title = 'Click on country to see amount of daily cases : '
).mark_bar(color = '#de425b', size = 1.5).encode(
    x = alt.X('Date:T', title = None),
    y = alt.Y('New_cases:Q', 
              title = None, 
              axis = alt.Axis(
                  format = '.1s',
                  )
              
             ),
    detail = alt.Detail('Country:N'),
)

Тепер поєднаємо ці графіки у складне представлення.
Верхній буде показувати розподіл сумарної кількості випадків на карті світу. А нижній зміну денної кількості випадків протягом двох років, для обраної країни.

Щоб користувач зрозумів, що для того щоб переглянути динаміку для певної країни на неї потрібно натиснути, я написала про це взаголовку. Так користувач одразу зверне на це свою увагу.

Вісь Y буде змінюватися залежно від обраної країни, це дозволить нам краще роздивитися динаміку для країн з маленькими значеннями.

Мінус такого представлення - на нижньому графіку важко дізнатися точну кількість випадків для певного дня. І користувачу потрібно знати приблизне розташування країни на карті, щоб подивитися для неї інформацію

In [ ]:
select_country = alt.selection_single(on = 'click', fields = ['Country'], nearest = False, empty = 'none')

In [ ]:
alt.vconcat(map_total_cases.properties(width = 900, height = 400).add_selection(select_country), 
            line_chart_total.properties(width = 900, height = 250).transform_filter(select_country)
).properties(background = '#F9F9F9', padding = 25
).configure_title(font = 'Times', 
                  fontSize = 18, 
                  dy = -20, 
                  anchor='middle'
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 12,
                   orient='none',
                   legendX=5, legendY=320,
                   direction='horizontal',
).configure_axis(ticks = False,
                 domain = False,
                 gridDash = [2, 2],
                 titleFont = 'Times',
                 labelFont = 'Times',
                 titleFontSize = 14,
                 labelFontSize = 12,
                 labelPadding = 5,
)

## (2) Інтерактивний графік який показує, як змінювалася кількість нових випадків зараження кожного дня впродовж 2-ох років.

Верхній графік буде відображати часовий проміжок, який користувач вибере на меншому нижньому графіку.

Я не стала підписувати тут вісі, бо назва графіку вже описує їх.

Я зробила верхній і нижній графік різних кольорів, щоб вони краще візуально розрізнялися. 

Вісь Y рухома, значення на ній міняються залежно він вибраного інтервалу. Це дозволить детальніше розглянути місця де значення Y дуже маленькі.

Кожен стовпчик на графіку ввідображає кількість випадків зараження за день у всьому світі (сума по країнах). Я обрала bar chart, бо за допомогою стовпчиків зручно передавати і порівнювати кількісні показники.

Я зробила кожен стовпчик досить маленького розміру, щоб вони не накладалися один на одного. 

In [ ]:
date_brush = alt.selection_interval(encodings=['x'])

In [ ]:
bar_chart_daily = alt.Chart(df, title = 'Daily number of Covid-19 infections (Global)'
).mark_bar(color = '#e06f45', size = 1.5).encode(
    x = alt.X('Date:T', title = None),
    y = alt.Y('New_cases:Q', 
              axis = alt.Axis(
                  format = '.2s',
                  ),
              aggregate = 'sum', 
              title = None
             )
)

bar_chart_daily_small = alt.Chart(df, title = 'Select interval'
).mark_bar(color = '#003f5c', size = 1.5).encode(
    x = alt.X('Date:T', title = None),
    y = alt.Y('New_cases:Q', 
              axis = alt.Axis(
                  format = '.2s',
                  ),
              aggregate = 'sum', 
              title = None
             )
)

alt.vconcat(bar_chart_daily.properties(width = 900, height = 400).transform_filter(date_brush), 
            bar_chart_daily_small.properties(width = 900, height = 100).add_selection(date_brush)
).properties(background = '#F9F9F9', 
             padding = 25
).configure_title(font = 'Times', 
                  fontSize = 18, 
                  anchor='middle'
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 14,
).configure_axis(ticks = False,
                 domain = False,
                 gridDash = [2, 2],
                 titleFont = 'Times',
                 labelFont = 'Times',
                 titleFontSize = 14,
                 labelFontSize = 12
)

## (3) Кількість випадків зараження залежно від регіону ВООЗ (всього 6 регіонів) та місяця

Кількісні значення для різних регіонів позначені різними кольорами. Щоб подивитися дані лише для якогось певного регіону, можна обрати його на легенді. За замовчуванням всі регіони вважаються виділеними.

На графіку передається сума випадків захворюваності за місяць. 


Щоб дінатися точну кількість випадків захворювання у певний час для окремого регіону, можна скористатися Tooltip (підказка про це взаголовоку).

Сегменти всередині кожної колонки посортовані, так ми зможемо одразу визначити у якому регіоні за певний місяць, було найбільше випадків захворювання, а в якому найменше.

Мінуси цього представлення :
 - Рухома вісь Y не дозволяє порівняти регіони між собою (але так ми можемо ближче роздивитися маленькі значення)
 - в bar chart якщо дивитися на всі регіони одночасно, дані для кожного з них зміщені. Щоб дізнатися точне число випадків захворювання, потрібно переключитися на окремий регіон, або скористатися Tooltip.

In [ ]:
select_legend_region = alt.selection_single(fields = ['WHO_region'], bind = 'legend', nearest = False, empty = 'all', name = 'Select')

alt.Chart(df, 
          title={"text": ["Month number of Covid-19 infections depending on WHO region"], 
                 "subtitle": ["(hover over the required column for more information / select region on legend)"],
                 "subtitleColor": '#333333'}
).mark_bar().encode(
    x = alt.X('month(Date):O', title = None),
    y = alt.Y('New_cases:Q', 
              aggregate = 'sum', 
              title = None,
              axis = alt.Axis(
                  format = '.1s',
                  )),
    color = alt.Color('WHO_region:N',
                      title = 'select WHO region',
                      scale = alt.Scale(
                          range = ["#003f5c", "#444e86", "#955196", "#dd5182", "#ff6e54", "#ffa600"],
                          domain = ['AFRO', 'AMRO', 'SEARO', 'EURO', 'EMRO', 'WPRO'],
                      )),
    order = alt.Order(field = 'New_cases', type = 'quantitative', aggregate = 'sum', sort = 'ascending'),
    tooltip = [
        alt.Tooltip('month(Date):O', title = 'Month'),
        alt.Tooltip('WHO_region:N', title = 'Region'),
        alt.Tooltip('New_cases:Q', aggregate = 'sum', title = 'Number of cases'),
        
    ],
).properties(width = 800, 
             height = 400,
             background = '#F9F9F9', 
             padding = 25
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 13,
                   symbolType = 'circle', 
                   symbolOpacity = 1.0 
).configure_title(font = 'Times', 
                  fontSize = 18, 
                  dy = -10, 
                  anchor='middle'
).configure_axis(ticks = False,
                 domain = False,
                 gridDash = [3, 3],
                 titleFont = 'Times',
                 labelFont = 'Times',
                 labelPadding = 10,
                 titleFontSize = 14,labelAngle = 0
).add_selection(select_legend_region
).transform_filter(select_legend_region
).configure_view(strokeWidth = 0)

## (4) Рівень захворюваності в Україна у розрізі місяців та років

In [ ]:
Ukraine_df = df[df.Country == 'Ukraine']
Ukraine_df.head()

На даному графіку порівнюється :
 - рівень захворюваності у 2020 та у 2021
 - рівень захворюваності у різні місяці (на графіку показана максимальна кількість випадків за день, яка зустрічалася у вказаний місяць)
 
Для того, щоб передати кількість випадків інфікування, я використала bar chart, стовпчиками буде зручно передавати кількісні показники і потім візуально їх порівнювати.

Я вирішила відображати максимальне значення денних випадків інфікування, яке зустрічалося за місяць. Як альтернативу, можна було б передавати середнє значення за місяць або ж суму. Але мені здалось що було б цікаво побачити саме "стелю" кожного місяця.

Щоб порівняти між собою 2020 і 2021 роки, я використала кольори. Стовпчики для 2020 подані темним синім кольором, а для 2021 - жовтим. Ці кольори доситатньо відрізняються один від одного, тому на графіку буде легко розмежувати 2020 і 2021 роки.

Я забрала підписи років з осі X, тому що це можна дізнатися на легенді, і в нас тільки два роки. 

Також я додала selection, щоб на легенді можна було виділити значення тільки для одного року, і порівняти значення у різні місяці.

Також я додала Tooltip, щоб можна було подивитися на точний числовий показик при наведенні на колонку. І підказку про це в підзаголовок.

In [ ]:
input_radio = alt.binding_radio(options = Ukraine_df.Year.unique())
select_year = alt.selection_single(fields = ['Year'], bind = 'legend', name = 'Select year', empty = 'all')

In [ ]:
alt.Chart(Ukraine_df,
          title={"text": ["The incidence of Covid-19 in Ukraine in terms of months and years"], 
                 "subtitle": ["(hover over the required column for more information / select year on Legend)"],
                 "subtitleColor": '#333333'}
).mark_bar().encode(
    y = alt.Y('New_cases:Q',
              aggregate = 'max',
              axis = alt.Axis(
                  format = '.1s',
                  ),
              title = 'Max number of cases per day in month', # "the maximum value of cases per day during the month"
             ),
    x = alt.X('year(Date):O', title = None, axis = None),
    color = alt.Color('Year:O', 
                      title = 'select Year',
                      scale = alt.Scale(
                          range = ["#003f5c", "#ffa600"],
                          domain = [2020, 2021],
                      )),
    column = alt.Column('month(Date):O', title = None),
    tooltip = [
        alt.Tooltip('year(Date):O', aggregate = 'max', title = 'year'),
        alt.Tooltip('month(Date):O', aggregate = 'max', title = 'month'),
        alt.Tooltip('New_cases:Q', aggregate = 'max', title = 'number of cases')
    ],
    opacity = alt.condition(
        select_year,
        alt.value(1.0),
        alt.value(0.0)
    )
).add_selection(
    select_year
).properties(width = 500/12, height = 400, background = '#F9F9F9', padding = 25
).configure_title(font = 'Times', 
                  fontSize = 18, 
                  dy = -20, 
                  anchor='middle'
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 14,
).configure_axis(ticks = False,
                 domain = False,
                 gridDash = [3, 3],
                 titleFont = 'Times',
                 labelFont = 'Times',
                 titleFontSize = 14,
).configure_view(strokeWidth = 0)

---- **Альтернатива**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for m in range(1, 13):
    max_2020 = Ukraine_df.loc[(Ukraine_df['Date'].dt.month == m) & (Ukraine_df['Date'].dt.year == 2020)]['New_cases'].max()
    Ukraine_df.loc[((Ukraine_df['Date'].dt.month == m) & (Ukraine_df['Date'].dt.year == 2020)), 'Max_2020'] = max_2020
    
    max_2021 = Ukraine_df.loc[(Ukraine_df['Date'].dt.month == m) & (Ukraine_df['Date'].dt.year == 2021)]['New_cases'].max()
    Ukraine_df.loc[((Ukraine_df['Date'].dt.month == m) & (Ukraine_df['Date'].dt.year == 2021)), 'Max_2021'] = max_2021
    
Ukraine_df.head()

Як альтернативу Bar Chart, я вирішила використати гантельковий графік. По осі Y йдуть місяці, по осі X - кількість випадків захворюваності. Роки, як і в попередньому графіку, передаються кольорами.

Цей графік зручний, для порівняння між собою двох років, завдяки лінії ми можемо бачити різницю між 2020 і 2021 роком.

Мінус: якщо різниця між роками маленька, то цієї лінії не видно на графіку.

In [ ]:
lines = alt.Chart(Ukraine_df, 
                  title={"text": ["The incidence of Covid-19 in Ukraine in terms of months and years"], 
                 "subtitle": ["(hover over the required dot for more information)"],
                 "subtitleColor": '#333333'}
).mark_line().encode(
    x = alt.X('max(Max_2020):Q', ),
    x2 = alt.X2('max(Max_2021):Q', ),
    y = alt.Y('month(Date):O', ))

points = alt.Chart(Ukraine_df).mark_circle(size = 100).encode(
    x = alt.X('New_cases:Q', 
              aggregate = 'max', 
              title = 'Max number of cases per day in month',
              axis = alt.Axis(
                  format = '.2s',
                  ),),
    y = alt.Y('month(Date):O', title = None),
    color  = alt.Color('year(Date):O', title = 'Year',
                       scale = alt.Scale(
                          range = ["#003f5c", "#ffa600"],
                          domain = [2020, 2021],
                      )),
    tooltip = [
        alt.Tooltip('year(Date):O', aggregate = 'max', title = 'year'),
        alt.Tooltip('month(Date):O', aggregate = 'max', title = 'month'),
        alt.Tooltip('New_cases:Q', aggregate = 'max', title = 'number of cases')
    ],
)

alt.layer(lines, points).properties(width = 800, height = 600, background = '#F9F9F9', padding = 25
).configure_title(font = 'Times', 
                  fontSize = 18, 
                  dy = -20, 
                  anchor='middle'
).configure_legend(titleFont = 'Times',
                   labelFont = 'Times',
                   titleFontSize = 14,
                   labelFontSize = 14,
).configure_axis(ticks = False,
                 domain = False,
                 gridDash = [3, 3],
                 titleFont = 'Times',
                 labelFont = 'Times',
                 titleFontSize = 14,
                 labelFontSize = 13,
                 labelPadding = 10,
                 titlePadding = 10
).configure_view(strokeWidth = 0)